# Run macro file 

This notebook shows how load an ImageJ macrofile linked to a dataset and analyze all the images contained in that dataset with the macro.
This notebook assumes that a macro file has been linked to the dataset. If more than one macro is linked, the first one found will be applied.

Fiji has been installed with few other plugins. For more information, check the Dockerfile.

In [11]:
//Add dependencies to the classpath
%classpath add jar /opt/java-apps/Fiji.app/jars/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/*
%classpath add jar /opt/java-apps/Fiji.app/plugins/OMERO.imagej-5.5.6/*

### Description

The following section shows
 * how to load a macro file linked to a dataset
 * how to analyze all the images in the dataset using the macro
 

In [12]:
g = new EasyForm("Enter credentials and continue to the next cell. Do not re-run this cell")
g.addTextField("Server").onInit({g['Server'] = "outreach.openmicroscopy.org"})
g.addTextField("UserName")
g.addPasswordField("Password")
g


### Connect to OMERO

In [13]:
import omero.gateway.Gateway
import omero.gateway.LoginCredentials
import omero.log.SimpleLogger

// Method to connect to OMERO
def connect_to_omero() {
    "Connect to OMERO"

    credentials = new LoginCredentials()
    credentials.getServer().setHostname(g['Server'])
    credentials.getServer().setPort(4064)
    credentials.getUser().setUsername(g['UserName'].trim())
    credentials.getUser().setPassword(g['Password'].trim())
    simpleLogger = new SimpleLogger()
    gateway = new Gateway(simpleLogger)
    gateway.connect(credentials)
    return gateway

}

// Connect to OMERO
println "connecting..."
gateway = connect_to_omero()
println "connected..."


connecting...
connected...


null

In [14]:
ga = new EasyForm("Select the Dataset to analyze. Do not re-run this cell")
ga.addTextField("DatasetID")
ga

### Run the macro

In [16]:
import java.util.ArrayList
import java.lang.Math
import java.lang.StringBuffer
import java.nio.ByteBuffer
import java.io.File
import java.io.PrintWriter

import java.nio.file.Files

import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.DataManagerFacility
import omero.gateway.facility.MetadataFacility
import omero.gateway.facility.ROIFacility
import omero.gateway.model.FileAnnotationData
import omero.gateway.model.DatasetData
import omero.gateway.model.ImageData

import omero.model.DatasetI
import omero.model.ImageI
import omero.log.SimpleLogger
import omero.model.ChecksumAlgorithmI
import omero.model.FileAnnotationI
import omero.model.OriginalFile
import omero.model.OriginalFileI
import omero.model.enums.ChecksumAlgorithmSHA1160

import static omero.rtypes.rlong
import static omero.rtypes.rstring


import org.openmicroscopy.shoola.util.roi.io.ROIReader

import ij.IJ
import ij.plugin.frame.RoiManager
import ij.measure.ResultsTable

//Connection information
HOST = g['Server']
USERNAME = g['UserName']
PASSWORD = g['Password']

PORT = 4064
//size of the chunk to load
INC = 262144
//Dataset to analyze Convert the String as long
dataset_id = ga['DatasetID'].toLong()


// Helper methods
def get_images(gateway, dataset_id) {
    "List all image's ids contained in a Dataset"
    
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    browse = gateway.getFacility(BrowseFacility)
    ids = new ArrayList(1)
    ids.add(new Long(dataset_id))
    return browse.getImagesForDatasets(ctx, ids)
}

def open_image_plus(HOST, USERNAME, PASSWORD, PORT, group_id, image_id) {
    "Open the image using the Bio-Formats Importer"

    StringBuilder options = new StringBuilder()
    options.append("location=[OMERO] open=[omero:server=")
    options.append(g['Server'])
    options.append("\nuser=")
    options.append(g['UserName'].trim())
    options.append("\nport=")
    options.append(4064)
    options.append("\npass=")
    options.append(g['Password'].trim())
    options.append("\ngroupID=")
    options.append(group_id)
    options.append("\niid=")
    options.append(image_id)
    options.append("] ")
    options.append("windowless=true view=Hyperstack ")
    IJ.runPlugIn("loci.plugins.LociImporter", options.toString())
}

// Load the macro file
def get_macro_file(gateway, dataset_id) {
    
    user = gateway.getLoggedInUser()
    ctx = new SecurityContext(user.getGroupId())
    svc = gateway.getFacility(MetadataFacility)
    //Find the dataset
    browse = gateway.getFacility(BrowseFacility)
    object = browse.findIObject(ctx, "omero.model.Dataset", dataset_id)
    dataset = new DatasetData(object);
    users = new ArrayList(1)
    users.add(new Long(user.getId()))
    types = new ArrayList(1)
    types.add(FileAnnotationData.class)
    annotations = svc.getAnnotations(ctx, dataset, types, users)
    
    store = gateway.getRawFileService(ctx);
    querySvc = gateway.getQueryService(ctx)
    index = 0
    annotations.each { fa ->
        of = (OriginalFile) querySvc.get("omero.model.OriginalFile", fa.getFileID());
        name = of.getName().getValue()
        //load the first annotation with ijm
        if (name.endsWith(".ijm") && index == 0) {
            file = File.createTempFile(name, ".ijm")
            stream = new FileOutputStream(file)
            store.setFileId(fa.getFileID());
            offset = 0;
            size = of.getSize().getValue();
            try {
                for (offset = 0; (offset+INC) < size;) {
                    stream.write(store.read(offset, INC));
                    offset += INC;
                }
            } finally {
                stream.write(store.read(offset, (int) (size-offset)));
                stream.close();
            }
            index++
        }
    }
    store.close()
    return file
}

// Prepare and save the results table
def upload_csv_to_omero(ctx, file, image_id) {
    "Upload the CSV file and attach it to the specified project"
    svc = gateway.getFacility(DataManagerFacility)

    file_size = file.length()
    original_file = new OriginalFileI()
    original_file.setName(rstring(file.getName()))
    original_file.setPath(rstring(file.getAbsolutePath()))
    original_file.setSize(rlong(file_size))
    checksum_algorithm = new ChecksumAlgorithmI()
    checksum_algorithm.setValue(rstring(ChecksumAlgorithmSHA1160.value))
    original_file.setHasher(checksum_algorithm)
    original_file.setMimetype(rstring("text/csv"))
    original_file = svc.saveAndReturnObject(ctx, original_file)
    store = gateway.getRawFileService(ctx)

    // Open file and read stream
    INC = 262144
    pos = 0
    buf = new byte[INC]
    ByteBuffer bbuf = null
    stream = null
    try {
        store.setFileId(original_file.getId().getValue())
        stream = new FileInputStream(file)
        while ((rlen = stream.read(buf)) > 0) {
            store.write(buf, pos, rlen)
            pos += rlen
            bbuf = ByteBuffer.wrap(buf)
            bbuf.limit(rlen)
        }
        original_file = store.save()
    } finally {
        if (stream != null) {
            stream.close()
        }
        store.close()
    }
    // create the file annotation
    namespace = "training.demo"
    fa = new FileAnnotationI()
    fa.setFile(original_file)
    fa.setNs(rstring(namespace))

    data_object = new ImageData(new ImageI(image_id, false)) 
    svc.attachAnnotation(ctx, new FileAnnotationData(fa), data_object)
}

// Main part of the script

exp = gateway.getLoggedInUser()
group_id = exp.getGroupId()
exp_id = exp.getId()

// Load all the images contained in the dataset
println "loading images..."
images = get_images(gateway, dataset_id)

// Load the file annotations linked to the dataset.
println "loading macro..."
macro_file = get_macro_file(gateway, dataset_id)

//Apply the macro to all the images in the dataset
println "analyzing images..."

tmp_dir = Files.createTempDirectory("Fiji_csv")
path = tmp_dir.resolve("idr0021_merged_results.csv")
file_path = Files.createFile(path)
file = new File(file_path.toString())

images.each { img ->
    // if target_user ~= None:
    // Switch context to target user and open omeroImage as ImagePlus object
    img_id = img.getId()
    
    imp = open_image_plus(HOST, USERNAME, PASSWORD, PORT, String.valueOf(group_id), String.valueOf(img_id))
    println "Image opened: "+img_id
    imp = IJ.getImage()
    println "Running macro on image: "+img_id
    IJ.runMacroFile(macro_file.getAbsolutePath())

    // Save the ROIs
    reader = new ROIReader()
    roi_list = reader.readImageJROIFromSources(img_id, imp)
    if (roi_list.size() > 0) {
        println "saving ROIs"
        roi_facility = gateway.getFacility(ROIFacility)
        result = roi_facility.saveROIs(ctx, img_id, exp_id, roi_list)
    }
    
    path = tmp_dir.resolve(img.getName()+"_results.csv")
    file_path = Files.createFile(path)
    file = new File(file_path.toString())
    //Save the results
    value = reader.readResults(file)
    if (value) { //some results to upload
        println "saving Results as CSV"
        upload_csv_to_omero(ctx, file, img_id)
    }
    imp.changes = false     /// Prevent "Save Changes?" dialog
    imp.close()
    IJ.run("Close All")
}

// delete the local copy of the temporary file and directory
dir = new File(tmp_dir.toString())
entries = dir.listFiles()
for (i = 0; i < entries.length; i++) {
    entries[i].delete()
}
dir.delete()

// Close the connection
gateway.disconnect()
//Delete macrofile
macro_file.delete()
println "processing done"

loading images...
loading macro...
analyzing images...
Image opened: 9542
Running macro on image: 9542
 	Area	Mean	Min	Max
1	25.898	3.452	0	7
2	0.028	6.111	5	7
3	0.024	6.267	4	7
4	0.268	6.117	2	7
5	0.336	5.565	2	7
6	0.063	5.975	3	7
7	0.027	6.176	4	7
8	0.038	6.000	4	7
9	0.103	5.561	2	7
10	0.056	6.083	4	7
11	0.045	5.966	4	7
12	0.052	6.182	4	7
13	0.078	5.760	3	7
14	0.027	5.059	3	7
15	0.017	5.545	4	7
16	0.077	5.959	4	7
17	0.083	6.151	4	7
18	0.140	5.899	2	7
19	0.030	5.947	4	7
20	0.036	5.826	4	7
21	0.464	5.530	1	7
22	0.072	6.022	4	7
23	0.502	5.547	1	7
24	0.025	6.438	5	7
25	0.019	6.250	5	7
26	0.019	6.667	6	7
27	0.093	5.932	4	7
28	0.016	6.100	4	7
29	0.020	5.231	2	7
30	0.064	6.317	4	7
31	0.113	5.861	3	7
32	0.030	6.053	4	7
33	0.061	5.974	4	7
34	0.039	6.000	5	7
35	0.220	6.121	3	7
36	0.020	6.615	5	7
37	0.041	5.423	3	7
38	0.034	5.182	3	7
39	0.411	5.672	1	7
40	0.017	6.818	6	7
41	0.016	6.600	5	7
42	0.045	6.552	5	7
43	0.025	5.250	3	7
44	0.050	6.219	4	7
45	0.061	6.179	3	7
46	0.016	6.300	5	7
47	0.020	5.

null